# CS 109A Data Science: Project
**Harvard University**<br>
**Fall 2016**<br>
**Team member:** Becerra, Fernando; Mayo, Andrew Wayne; Ding, Yi; Cao, Xinrui<br>
**Project topic:** Predicting Seizures and Epilepsy

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import time
from scipy.stats import mode
from sklearn import linear_model
import sklearn.linear_model as sk
import matplotlib
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
import sklearn.discriminant_analysis as da
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

%matplotlib inline

# Cleaning

In [2]:
# Read in data set UKDA-2024-tab
UKDA16 = pd.read_table('./datasets/NCDS/ncds0123.tab')

/Users/fbecerra/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,2,3,4,5,1756,1757,1758,1759,1760,1761,1762,1763,1764) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#sweep 1
# reason for special education
a = (UKDA16['n390'] =='10') | (UKDA16['n391'] =='10') | (UKDA16['n392'] =='10')
# Epileptic condition
b = (UKDA16['n415'] =='3') | (UKDA16['n415'] =='4')| (UKDA16['n415'] =='5') | (UKDA16['n415'] =='6')
# Fit or convulsion in 1st year of life
c = (UKDA16['n274'] == '2')
# Fit or convulsion after 1st year
d = (UKDA16['n275'] == '2')
# Epilepsy identification
e = (UKDA16['n1842']=='5')
s1 =a|b|c|d|e

In [4]:
# sweep 2
# Has child had epilepsy attacks
a = (UKDA16['n1307']=='1')|(UKDA16['n1307']=='2')|(UKDA16['n1307']=='3')|(UKDA16['n1307']=='4')|(UKDA16['n1307']=='5')
# Age at most recent epilepsy attack
UKDA16['n1314']
# Age at 1st epilepsy attack
UKDA16['n1317']
# Reason for special education
b = (UKDA16['n1477'] =='7') | (UKDA16['n1478'] =='7') | (UKDA16['n1479'] =='7')
# Reason for absence from school
c = (UKDA16['n1321']=='8') |(UKDA16['n1322']=='8')|(UKDA16['n1323']=='8')|(UKDA16['n1324']=='8')|(UKDA16['n1325']=='8') 
# Reason for prescription
d = (UKDA16['n1337']=='1') |(UKDA16['n1338']=='1')
# Ever seen specialist-convulsions,fits
e = (UKDA16['n1394']=='2')|(UKDA16['n1394']=='3')|(UKDA16['n1394']=='5')
# Has child ever had convulsions
f = (UKDA16['n1502'] =='2')| (UKDA16['n1502'] =='3')|(UKDA16['n1502'] =='4')
s2 = a|b|c|d|e
s1s2 =s1|s2

In [5]:
# sweep 3
# Type hcap for which will require help
a = (UKDA16['n2416'] =='7')
# Nature of child-s disability-
b = (UKDA16['n2663'] =='7')|(UKDA16['n2664'] =='7')|(UKDA16['n2665'] =='7')|(UKDA16['n2666'] =='7')|(UKDA16['n2667'] =='7')
# Category of child's handicap
c = (UKDA16['n1893'] =='8')|(UKDA16['n1894'] =='8')|(UKDA16['n1895'] =='8')
# Reason for hosp admiss last 12 mnths
d = (UKDA16['n1904'] =='17')
# Reason hosp outpatient last yr
e = (UKDA16['n1910'] =='17')
# Epilepsy
#f = (UKDA16['n2032'] =='17')
# Reason for absence from school
g = (UKDA16['n2554'] =='11')|(UKDA16['n2556'] =='11')|(UKDA16['n2557'] =='11')|(UKDA16['n2558'] =='11')|(UKDA16['n2559'] =='11')
# Hospital admissions
h = (UKDA16['n2598'] =='7') | (UKDA16['n2599'] =='7') | (UKDA16['n2600'] =='7') | (UKDA16['n2601'] =='7') 
# Hosp outpatient,diagnosed cond
j = (UKDA16['n2603'] =='10')|(UKDA16['n2604'] =='10')|(UKDA16['n2605'] =='10')|(UKDA16['n2606'] =='10')|(UKDA16['n2607'] =='10')|(UKDA16['n2608'] =='10')
# Convulsions,fits-unconsciousness
k = (UKDA16['n2614'] =='1')
# When convulsions,fits 1st occured
(UKDA16['n2615'] =='5')|(UKDA16['n2615'] =='5')
# Convulsions-most recent occurrence
l = (UKDA16['n2616'] =='1')|(UKDA16['n2616'] =='2')|(UKDA16['n2616'] =='3')|(UKDA16['n2616'] =='4')
s3 = d|e|g|h|j|l

- From Sweep 1 and Sweep 2, we extract all children who had any sign of epilepsy and convulsion. 
- From Sweep 3, we extract all children who had any sign of epilepsy and convulsion after age 11 (eg. hospitalization for convulsion or epilepsy, recent occurence of convulsion within 7 years)
- We take the join of s1s2 and s3, which include all children who have sign of epilepsy before 11 and had recurrence after 11.

In [6]:
sum(s1s2)

1026

In [7]:
sum(s1s2 & s3)

250

- select social economic status and medical status variables from three sweep and combine

In [8]:
filename = "./datasets/NCDS/sweep1.txt"
s1_variable = []
for line in open(filename):
    if 'Pos.' in line:
        name = line.rstrip().replace("\t",' ').split(' ')[5]
        s1_variable.append(name)
        
print len(s1_variable)

131


In [9]:
filename = "./datasets/NCDS/sweep2.txt"
s2_variable = []
for line in open(filename):
    if 'Pos.' in line:
        name = line.rstrip().replace("\t",' ').split(' ')[5]
        s2_variable.append(name)
        
print len(s2_variable)

46


In [10]:
filename = "./datasets/NCDS/sweep3.txt"
s3_variable = []
for line in open(filename):
    if 'Pos.' in line:
        name = line.rstrip().replace("\t",' ').split(' ')[5]
        s3_variable.append(name)
        
print len(s3_variable)

44


In [11]:
variables = s1_variable + s2_variable + s3_variable

### Here comes our final dataset!

In [12]:
data = UKDA16[variables][s1s2]

- Count missing rate

In [13]:
a = (data=='-1').sum(axis =0)/1046
b = (data==' ').sum(axis =0)/1046
sum(a+b<0.5)

195

In [14]:
#drop data with >0.5 missing rate
data = data[list(data.columns[np.where(a+b<0.5)])]

# fill in the blanks as '-1'
for col in data.columns:
    value = data[col]
    missing_index = (value == ' ')
    data[col][missing_index] = '-1'

data

,n622,n0region,n1region,n2region,n3region,n545,n490,n492,n524,n526,...,n2560,n2614,n2624,n2631,n2020,n2021,n2029,n2030,n2032,n2034
7,1,10,-1,10,10,4,3,6,-1,-1,...,7,2,1,5,1,1,1,1,1,1
25,1,2,10,10,10,4,5,3,1,4,...,7,2,1,1,1,1,1,1,1,1
34,1,9,8,8,8,4,4,4,-1,10,...,11,2,1,1,1,1,1,2,1,1
56,2,5,8,-1,-1,4,5,3,10,4,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
61,2,7,7,7,7,4,2,5,3,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
64,1,7,7,7,7,4,5,3,7,4,...,11,1,1,1,1,1,1,1,1,1
91,2,2,2,2,2,4,5,3,1,4,...,-1,-1,-1,-1,1,1,1,1,1,1
98,2,11,11,11,11,4,1,4,2,5,...,7,1,1,1,1,1,1,1,1,1
102,2,3,3,3,3,4,1,4,1,4,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
138,2,1,1,1,1,4,5,3,9,4,...,11,1,3,1,1,1,1,1,1,1


In [15]:
data = data.convert_objects(convert_numeric=True)
data.shape

/Users/fbecerra/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


(1026, 195)

In [16]:
# fill in the missing data
# randomly choose a value from the same colunm
# for col in data.columns:
#     value = data[col]
    
#     missing_index = (value == '-1') | (value == ' ')
#     missing_number = sum(missing_index)
#     filled_index = (value != ' ') & (value != '-1')
#     data[col][missing_index] = np.random.choice(value[filled_index], missing_number)

In [17]:
# create response
y = np.zeros(len(UKDA16))
y = pd.DataFrame(y, index = UKDA16.index)
y[s1s2&s3] = 1
y = y[s1s2]
y = y.astype(int)

# Model

Predictors are in `data`, response is in `y`


In [18]:
data

,n622,n0region,n1region,n2region,n3region,n545,n490,n492,n524,n526,...,n2560,n2614,n2624,n2631,n2020,n2021,n2029,n2030,n2032,n2034
7,1,10,-1,10,10,4,3,6,-1,-1,...,7,2,1,5,1,1,1,1,1,1
25,1,2,10,10,10,4,5,3,1,4,...,7,2,1,1,1,1,1,1,1,1
34,1,9,8,8,8,4,4,4,-1,10,...,11,2,1,1,1,1,1,2,1,1
56,2,5,8,-1,-1,4,5,3,10,4,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
61,2,7,7,7,7,4,2,5,3,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
64,1,7,7,7,7,4,5,3,7,4,...,11,1,1,1,1,1,1,1,1,1
91,2,2,2,2,2,4,5,3,1,4,...,-1,-1,-1,-1,1,1,1,1,1,1
98,2,11,11,11,11,4,1,4,2,5,...,7,1,1,1,1,1,1,1,1,1
102,2,3,3,3,3,4,1,4,1,4,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
138,2,1,1,1,1,4,5,3,9,4,...,11,1,3,1,1,1,1,1,1,1


In [19]:
y

,0
7,0
25,0
34,0
56,0
61,0
64,1
91,0
98,1
102,0
138,1


In [20]:
y = y[0] # This is required so that y has the right shape

In [21]:
def calculate_model(x_train, y_train, x_test, y_test, x, model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x)
    score_train = model.score(x_train, y_train)
    score_test = model.score(x_test, y_test)
    return score_train, score_test, y_pred

def print_info(title, train, test, cm):
    print '*** ' + title + ' ***'
    print 'Training score: %.3f, Testing score: %.3f' %(train.mean(), test.mean())
    print 'Non-relapse accuracy: %.3f, relapse accuracy: %.3f' %(float(cm[0, 0]) / cm[0].sum(), float(cm[1, 1]) / cm[1].sum())

In [22]:
n_iter = 10
n_rows = data.shape[0]
depths = range(1,8)
n_depths = len(depths)
#Cs = [0.1, 1, 10, 100]
#n_Cs = len(Cs)
C = 10

logreg_test = np.zeros(n_iter)
logreg_train = np.zeros(n_iter)
logreg_pred = np.zeros(n_rows)
dt_test = np.zeros((n_iter, n_depths))
dt_train = np.zeros((n_iter, n_depths))
dt_pred = np.zeros((n_rows, n_depths))
lda_test = np.zeros(n_iter)
lda_train = np.zeros(n_iter)
lda_pred = np.zeros(n_rows)
qda_test = np.zeros(n_iter)
qda_train = np.zeros(n_iter)
qda_pred = np.zeros(n_rows)
rf_test = np.zeros((n_iter, n_depths))
rf_train = np.zeros((n_iter, n_depths))
rf_pred = np.zeros((n_rows, n_depths))
svc_linear_test = np.zeros(n_iter)
svc_linear_train = np.zeros(n_iter)
svc_linear_pred = np.zeros(n_rows)
svc_quad_test = np.zeros(n_iter)
svc_quad_train = np.zeros(n_iter)
svc_quad_pred = np.zeros(n_rows)


for i in range(n_iter):

    print 'Iteration %i' %i

    x_train, x_test, y_train, y_test = train_test_split(data, y, train_size=0.6)

    logreg = LogisticRegression(C=1000)
    logreg_train[i], logreg_test[i], y_pred = calculate_model(x_train, y_train, x_test, y_test, data, logreg)
    logreg_pred += y_pred

    lda = LDA()
    lda_train[i], lda_test[i], y_pred = calculate_model(x_train, y_train, x_test, y_test, data, lda)
    lda_pred += y_pred

    qda = QDA()
    qda_train[i], qda_test[i], y_pred = calculate_model(x_train, y_train, x_test, y_test, data, qda)
    qda_pred += y_pred

    for depth in depths:

        dt = DecisionTree(max_depth = depth)
        dt_train[i, depth-1], dt_test[i, depth-1], y_pred = calculate_model(x_train, y_train, x_test, y_test, data, dt)
        dt_pred[:, depth-1] += y_pred

        rf = RandomForest(n_estimators=15, max_depth = depth)
        rf_train[i, depth-1], rf_test[i, depth-1], y_pred = calculate_model(x_train, y_train, x_test, y_test, data, rf)
        rf_pred[:, depth-1] += y_pred


    svc_linear = SVC(C=C, kernel='linear') #, class_weight=weights)
    svc_linear_train[i], svc_linear_test[i], y_pred = calculate_model(x_train, y_train, x_test, y_test, data, svc_linear)
    svc_linear_pred += y_pred 

    svc_quad = SVC(C=C, kernel='rbf') #, class_weight=weights)
    svc_quad_train[i], svc_quad_test[i], y_pred = calculate_model(x_train, y_train, x_test, y_test, data, svc_quad)
    svc_quad_pred += y_pred 

# Choose depth with maximum test score
dt_idx = np.argmax(dt_test.mean(axis=0))
rf_idx = np.argmax(rf_test.mean(axis=0))

# Choose C with maximum test score
#svc_linear_idx = np.argmax(svc_linear_test.mean(axis=0))
#svc_quad_idx = np.argmax(svc_quad_test.mean(axis=0))

# Take majority vote
logreg_pred = [1 if pred > n_iter/2. else 0 for pred in logreg_pred]
lda_pred = [1 if pred > n_iter/2. else 0 for pred in lda_pred]
qda_pred = [1 if pred > n_iter/2. else 0 for pred in qda_pred]
dt_pred = [1 if pred > n_iter/2. else 0 for pred in dt_pred[:, dt_idx]]
rf_pred = [1 if pred > n_iter/2. else 0 for pred in rf_pred[:, rf_idx]]
svc_linear_pred = [1 if pred > n_iter/2. else 0 for pred in svc_linear_pred]
svc_quad_pred = [1 if pred > n_iter/2. else 0 for pred in svc_quad_pred]

# Calculate accuracies
logreg_cm = confusion_matrix(y, logreg_pred)
lda_cm = confusion_matrix(y, lda_pred)
qda_cm = confusion_matrix(y, qda_pred)
dt_cm = confusion_matrix(y, dt_pred)
rf_cm = confusion_matrix(y, rf_pred)
svc_linear_cm = confusion_matrix(y, svc_linear_pred)
svc_quad_cm = confusion_matrix(y, svc_quad_pred)

print_info('LogReg', logreg_train, logreg_test, logreg_cm)
print_info('LDA', lda_train, lda_test, lda_cm)
print_info('QDA', qda_train, qda_test, qda_cm)
print_info('DT', dt_train[:, dt_idx], dt_test[:, dt_idx], dt_cm)
print 'Depth ', depths[dt_idx]
print_info('RF', rf_train[:, rf_idx], rf_test[:, rf_idx], rf_cm)
print 'Depth ', depths[rf_idx]
print_info('SVC linear', svc_linear_train, svc_linear_test, svc_linear_cm)
print_info('SVC quad', svc_quad_train, svc_quad_test, svc_quad_cm)

Iteration 0


/Users/fbecerra/anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
*** LogReg ***
Training score: 1.000, Testing score: 0.790
Non-relapse accuracy: 0.994, relapse accuracy: 0.940
*** LDA ***
Training score: 0.946, Testing score: 0.835
Non-relapse accuracy: 0.947, relapse accuracy: 0.904
*** QDA ***
Training score: 0.975, Testing score: 0.755
Non-relapse accuracy: 1.000, relapse accuracy: 0.520
*** DT ***
Training score: 0.929, Testing score: 0.919
Non-relapse accuracy: 0.909, relapse accuracy: 0.992
Depth  3
*** RF ***
Training score: 0.965, Testing score: 0.868
Non-relapse accuracy: 0.978, relapse accuracy: 0.876
Depth  6
*** SVC linear ***
Training score: 1.000, Testing score: 0.788
Non-relapse accuracy: 0.991, relapse accuracy: 0.936
*** SVC quad ***
Training score: 1.000, Testing score: 0.756
Non-relapse accuracy: 1.000, relapse accuracy: 0.648


Non-relapse and relapse accuracies are based on the *whole* dataset -- It is basically a misclassification rate. To calculate the prediction we use a majority vote based on `n_iter = 10`, just like we did in previous homework. For the case of decision tree and random forest, we also calculate the optimal depth.